In [ ]:
# Directorios para datos.

# LOCAL
data_dir_path = ""

#4. Entrenamiento de modelos de NER con Stanza y spaCy a partir de ficheros en notación BRAT

Tanto Stanza como spaCy permiten entrenar modelos de detección de entidades a partir de corpus etiquetados. El formato que utiliza cada librería es distinto y se va a partir de una pequeña porción del dataset Symptemist https://zenodo.org/records/10635215 para la detección de síntomas en documentos clínicos en español.

In [ ]:
!pip install stanza
!pip install spacy

##4.1 Descargamos el dataset en formato BRAT

El formato **BRAT** (BRAT Rapid Annotation Tool) es un formato de texto utilizado para la anotación de entidades, relaciones y eventos en corpus lingüísticos. Es ampliamente utilizado en tareas de Procesamiento del Lenguaje Natural (PLN) para etiquetar datos textuales.

Los datos anotados se almacenan en archivos de texto plano con extensión .txt para el texto original y .ann para las anotaciones. Así, habrá 2 ficheros con el mismo nombre y distinta extensión (.txt y .ann).

El archivo .ann contiene las anotaciones en un formato estructurado. A continuación se pone un ejemplo de uno de estos ficheros:


```
T1	SINTOMA 836 852	Hemograma normal
T2	SINTOMA 444 480	hematuria macroscópica postmiccional
T3	SINTOMA 498 512	microhematuria
```

El corpus a utilizar es uno reducido de Symptemist accesible en https://zenodo.org/records/8223654

Este corpus tiene anotadas entidades de tipo SINTOMA.


In [ ]:
# Descargamos el corpus de entreanmiento reducido de Symptemist https://zenodo.org/records/8223654
!wget -O symptemist_train.zip https://zenodo.org/records/8223654/files/symptemist_train.zip?download=1
# Descomprimimos el corpus
!unzip -n symptemist_train

--2025-02-26 16:14:24--  https://zenodo.org/records/8223654/files/symptemist_train.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.48.194, 188.185.43.25, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2905457 (2.8M) [application/octet-stream]
Saving to: ‘symptemist_train.zip’

symptemist_train.zi 100%[===================>]   2.77M   629KB/s    in 4.8s    

2025-02-26 16:14:29 (591 KB/s) - ‘symptemist_train.zip’ saved [2905457/2905457]

Archive:  symptemist_train.zip


##4.2 Entrenamiento del modelo NER en Stanza

Para esto tenemos que descargar los recursos de Stanza en español porque necesitamos usar el tokenizador de Stanza para transformar el formato BRAT a uno procesable por esta librería.

In [ ]:
# Descargamos el modelo en español de Stanza
import stanza

stanza.download('es')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: es (Spanish) ...
INFO:stanza:File exists: /root/stanza_resources/es/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


Para poder entrenar el NER de Stanza es necesario proporcionar un formato CoNLL basado en etiquetas BIO o BIOES.

Por lo que tenemos que incluir etiquetas BIO con su tipo de entidad como las siguientes ["B-ENFERMEDAD", "I-ENFERMEDAD", "B-FECHA"].

En la siguiente celda de código se muestra una función que realiza este proceso.

In [ ]:
# El conjunto de entrenamiento de Stanza utiliza el formato CoNLL
import stanza
import os


def brat_to_conll(brat_txt_path, brat_ann_path, output_conllu_path, stanza_tokenizer):
    # Cargar el texto original en fomato BRAT y lo transformamos a CoNLL
    # Es necesario mandar como parámetro un tokenizador de Stanza en el idioma del conjunto
    # de entrenamiento
    with open(brat_txt_path, "r", encoding="utf-8") as f:
        text = f.read()

    # Cargar las anotaciones BRAT
    entities = []
    with open(brat_ann_path, "r", encoding="utf-8") as f:
        for line in f:
            if line.startswith("T"):  # Ignorar relaciones y eventos
                parts = line.strip().split("\t")
                entity_info = parts[1].split()
                entity_type = entity_info[0]
                start = int(entity_info[1])
                end = int(entity_info[-1])
                entity_text = parts[2]
                entities.append((start, end, entity_type, entity_text))

    # Tokenizar el texto con Stanza y alinear entidades
    doc = stanza_tokenizer(text)

    # Mapear posiciones de caracteres a tokens
    token_offsets = []
    for sentence in doc.sentences:
        for token in sentence.tokens:
            token_offsets.append((token.start_char, token.end_char, token.text))

    # Asignar etiquetas BIO a cada token
    bio_tags = ["O"] * len(token_offsets)
    entities = sorted(entities, key=lambda x: x[0])  # Ordenar por posición de inicio

    for start, end, etype, _ in entities:
        entity_tags = []
        for i, (token_start, token_end, token_text) in enumerate(token_offsets):
            if token_start >= start and token_end <= end:
                entity_tags.append((i, etype))
            elif (token_start < end and token_end > start):  # Tokens parcialmente solapados
                entity_tags.append((i, etype))

        # Asignar etiquetas BIO
        if entity_tags:
            first = True
            for idx, tag in entity_tags:
                if first:
                    bio_tags[idx] = f"B-{tag}"
                    first = False
                else:
                    bio_tags[idx] = f"I-{tag}"

    # Escribir archivo CoNLL-U
    with open(output_conllu_path, "a", encoding="utf-8") as f:
        idx = 0
        for sentence in doc.sentences:
            for token in sentence.tokens:
                f.write(f"{token.text}\t{bio_tags[idx]}\n")
                idx += 1
            f.write("\n")


En la siguiente celda de código se cargan todos los ficheros de la carpeta del corpus Symptemist y usamos algunos ficheros para el conjunto de **train** (entrenamiento) entrenamiento y otros para el conjunto de **dev** (validación) que nos permitan entrenar el modelo. El conjunto de **dev** se utiliza para ir ajustando el modelo y no se corresponde con el conjunto de **test** (prueba) que sería algo ajeno al entrenamiento del problema.

Para reducir el tiempo de entrenamiento, seleccionamos menos ficheros.

In [ ]:
from sklearn.model_selection import train_test_split

brat_folder = "symptemist_train/subtask1-ner/brat"
filename_id = []
for filename in os.listdir(brat_folder):
    filename_id.append(filename.split(".")[0])

filename_id = list(set(filename_id)) # id unico
train_files, dev_files = train_test_split(filename_id, test_size=0.2, random_state=42)

# Reducimos para disminuir el tiempo de entrenamiento
train_files = train_files[:1000]
dev_files = dev_files[:200]


En la siguiente celda se guardan los dos ficheros en formato BIO utilizando las funciones anteriores.

In [ ]:
train_conll_path = "train.bio"
dev_conll_path = "dev.bio"

# Cargamos un tokenizador en español
nlp = stanza.Pipeline(lang="es", processors="tokenize")

for train_id in train_files:
    brat_txt_path = f"{brat_folder}/{train_id}.txt"
    brat_ann_path = f"{brat_folder}/{train_id}.ann"
    brat_to_conll(brat_txt_path, brat_ann_path, train_conll_path, nlp)

for dev_id in dev_files:
    brat_txt_path = f"{brat_folder}/{dev_id}.txt"
    brat_ann_path = f"{brat_folder}/{dev_id}.ann"
    brat_to_conll(brat_txt_path, brat_ann_path, dev_conll_path, nlp)


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Done loading processors!


Estanza permite transformar los ficheros en formato CoNLL con formato BIO en el formato JSON necesario para poder entrenar el modelo.

In [ ]:
import stanza.utils.datasets.ner.prepare_ner_file as prepare_ner_file

# Convertir al formato bio al formato json para el entrenamiento con Stanza
prepare_ner_file.process_dataset(train_conll_path, f"{train_conll_path}.json")
prepare_ner_file.process_dataset(dev_conll_path, f"{dev_conll_path}.json")

18990 examples loaded from train.bio
Generated json file train.bio.json
4662 examples loaded from dev.bio
Generated json file dev.bio.json


Una vez tenemos el corpus en un formato procesable por Stanza, lanzamos el script para realizar ese entrenamiento. Como se puede ver tiene distintos parámetros que se pueden configurar. Por ejemplo:

*   Número de pasos máximo   *--max_steps 1000*
*   El intervalo a partir del cual se evalúa el modelo   *--eval_interval 200*
*   Es recomendable utilizar un fichero preentrenado de embeddings  *--wordvec_pretrain_file*
*   Se pueden seleccionar distintos optimizadores (aquí utilizamos el adam)  *--optim adam*
*   Learning rate  *--lr 0.001*
*   Dropout  *--dropout 0.3*
*   El tamaño del batch  *--batch_size 32*



In [ ]:
new_model_stanza_dir = "stanza_ner_model"
new_model_stanza_file = "stanza_ner_trained.pt"

# Ejecutamos este comando con la configuración del entrenamiento.
# Hay que indicar un archivo preentrenado de vectores de palabras que se ha
# instalado al descargar el modelo en español usando el comando
# stanza.download('es')
!python -m stanza.models.ner_tagger \
  --train_file train.bio.json \
  --eval_file dev.bio.json \
  --mode train \
  --shorthand es_custom \
  --save_dir stanza_ner_model \
  --save_name stanza_ner_trained.pt \
  --max_steps 1000 \
  --eval_interval 200 \
  --wordvec_pretrain_file /root/stanza_resources/es/pretrain/conll17.pt \
  --optim adam \
  --lr 0.001 \
  --dropout 0.3 \
  --batch_size 32 \
  --no_char


2025-02-26 16:15:21 INFO: Running NER tagger in train mode
2025-02-26 16:15:21 INFO: ARGS USED AT TRAINING TIME:
batch_size: 32
bert_finetune: False
bert_hidden_layers: None
bert_learning_rate: 1.0
bert_model: None
char: False
char_dropout: 0
char_emb_dim: 100
char_hidden_dim: 100
char_lowercase: False
char_num_layers: 1
char_rec_dropout: 0
charlm: False
charlm_backward_file: None
charlm_forward_file: None
charlm_save_dir: saved_models/charlm
charlm_shorthand: None
connect_output_layers: False
data_dir: data/ner
device: cpu
dropout: 0.3
emb_finetune: True
emb_finetune_known_only: False
eval_file: dev.bio.json
eval_interval: 200
eval_output_file: None
finetune: False
finetune_load_name: None
gradient_checkpointing: False
hidden_dim: 256
ignore_tag_scores: None
input_transform: True
locked_dropout: 0.0
log_norms: False
log_step: 20
lora_alpha: 128
lora_dropout: 0.1
lora_modules_to_save: []
lora_rank: 64
lora_target_modules: ['query', 'value', 'output.dense', 'intermediate.dense']
lowerca

Al utilizar un modelo de embeddings preentrenado, debemos copiarlo porque para usar el modelo es necesario indicar ese modelo de embeddings preentreenados.

In [ ]:
# Una vez entrenado el modelo, necesitamos copiar el fichero wordvec_pretrain_file
# a la carpeta del modelo para poder usarlo de cara a la inferencia
!cp /root/stanza_resources/es/pretrain/conll17.pt stanza_ner_model/

###Inferencia

Una vez que hemos entrenado el modelo podemos realizar una inferencia para ver que identifica las nuevas entidades entrenadas.

Para eso descaregamos el fichero *P4_frases_medicas.csv* para poder extraer los síntomas de las mismas.

In [ ]:
# Descargamos un csv con frases médicas para extraer los síntomas
#!wget -N --no-check-certificate https://valencia.inf.um.es/valencia-plne/P4_frases_medicas.csv

--2025-02-21 12:16:43--  https://valencia.inf.um.es/valencia-plne/P4_frases_medicas.csv
Resolving valencia.inf.um.es (valencia.inf.um.es)... 155.54.204.133
Connecting to valencia.inf.um.es (valencia.inf.um.es)|155.54.204.133|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘P4_frases_medicas.csv’ not modified on server. Omitting download.



In [ ]:
import pandas as pd
import stanza

nlp = stanza.Pipeline(
    lang="es",
    processors="tokenize,ner",  # u otros que desees
    ner_model_path=f"{new_model_stanza_dir}/{new_model_stanza_file}",
    ner_pretrain_path=f"{new_model_stanza_dir}/conll17.pt",
    download_method=None,  # Usar modelo local
)

# Carga el archivo CSV
try:
    df = pd.read_csv(data_dir_path + 't3_frases_medicas.csv')
except FileNotFoundError:
    print("Error: El archivo 't3_frases_medicas.csv' no se encontró.")
    exit()

# Inicializa nuevas columnas en el DataFrame con listas vacías
df['entidades_stanza'] = [[] for _ in range(len(df))]


# Procesa cada frase en el DataFrame
for index, row in df.iterrows():
    frase = row['texto']  # Suponiendo que la columna con las frases se llama 'texto'
    stanzaDoc = nlp(frase)

    entidades = []

    for ent in stanzaDoc.ents:
        entidades.append(ent.text)

    df.at[index, 'entidades_stanza'] = entidades

df.head(20)

INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package                 |
---------------------------------------
| tokenize  | combined                |
| mwt       | combined                |
| ner       | stanza_ner...trained.pt |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


,texto,entidades_stanza
0,El paciente presenta fiebre alta y escalofríos...,"[fiebre, escalofríos]"
1,El paciente presenta dolor de cabeza intenso a...,[dolor de cabeza intenso acompañado de sensibi...
2,"El paciente presenta tos seca persistente, por...",[tos seca persistente]
3,"El paciente presenta dificultad para respirar,...",[dificultad para respirar]
4,El paciente presenta fatiga extrema y debilida...,"[fatiga extrema, debilidad muscular]"
5,El paciente presenta dolor muscular generaliza...,"[dolor muscular generalizado, hidratación adec..."
6,"El paciente presenta congestión nasal severa, ...",[congestión nasal severa]
7,El paciente presenta pérdida del sentido del o...,[pérdida del sentido del olfato y del gusto]
8,El paciente presenta sudoración nocturna recur...,[sudoración nocturna]
9,"El paciente presenta dolor abdominal difuso, p...",[dolor abdominal difuso]


##4.3 Entrenamos ahora el modelo en spaCy

spaCy permite también entrenar modelos de detección de entidades. El formato necesario para realizar esto es diferente del modelo BRAT y el modelo CoNLL que necesita Stanza.

A continuación, se muestra el código para traducir el formato BRAT al formato necesario por spaCy.

In [ ]:
import sys
import spacy
from pathlib import Path
from collections import namedtuple
def access_dataset(anns, txts):
    """
    - anns: [string] with *.ann file content.
    - txts: [string] with *.txt file content.
    - return: [(string, [(integer, integer, string)])]
      with [(text, [(start, end, entity)])].
    """
    # Example
    example = [
        ("Tokyo Tower is 333m tall.", [(0, 11, "BUILDING")]),
    ]

    # Split ANN into lines
    anns = [ann.split("\n") for ann in anns]

    # Discard empty lines
    anns = [[line for line in ann if line != ""] for ann in anns]

    # Parse ANN
    anns = [[parse_brat(line) for line in ann] for ann in anns]

    # Convert to SpaCy
    dataset = []
    for txt, ann in zip(txts, anns):
        data = (txt, list({(brat.start, brat.end, brat.entity) for brat in ann if brat is not None}))
        dataset.append(data)
    return dataset

def parse_brat(line):
    """
    - line: string with .ann line.
    - return: (string, string, integer, integer, string)
      with (id, entity, start, end, text).
    """
    # Class
    Brat = namedtuple("Brat", "id entity start end text")

    # Access data
    if line.startswith("T"):  # Ignorar relaciones y eventos
      id_ese_text = line.replace("\t", ";").split(";")
      id     = id_ese_text[0]
      entity_info = id_ese_text[1].split()
      entity = entity_info[0]
      start = int(entity_info[1])
      end = int(entity_info[-1])
      text   = " ".join(id_ese_text[4:])

    # Structure data
      return Brat(str(id), str(entity), int(start), int(end), str(text))
    else:
      return None

def readfile(filename):
    try:
        with open(filename, "r") as f:
            return str(f.read())
    except:
        return ""


In [ ]:
# Settings
spacy_nlp = spacy.blank("es")
train_spacy_path = "train.spacy"
dev_spacy_path = "dev.spacy"

# Input
train_anns = [readfile(f"{brat_folder}/{train_id}.ann") for train_id in train_files]
train_txts = [readfile(f"{brat_folder}/{train_id}.txt") for train_id in train_files]

dataset_train = access_dataset(train_anns, train_txts)
DB  = spacy.tokens.DocBin()

# Program
for text, annotations in dataset_train:
  try:
    document      = spacy_nlp(text)
    spans         = [document.char_span(s, e, label = l) for s,e,l in annotations]
    document.ents = [span for span in spans if span is not None]
    DB.add(document)
  except:
    print("[WARNING] one discarded")

# Output
DB.to_disk(train_spacy_path)

# Input
dev_anns = [readfile(f"{brat_folder}/{dev_id}.ann") for dev_id in dev_files]
dev_txts = [readfile(f"{brat_folder}/{dev_id}.txt") for dev_id in dev_files]

dataset_dev = access_dataset(dev_anns, dev_txts)
DB  = spacy.tokens.DocBin()

# Program
for text, annotations in dataset_dev:
  try:
    document      = spacy_nlp(text)
    spans         = [document.char_span(s, e, label = l) for s,e,l in annotations]
    document.ents = [span for span in spans if span is not None]
    DB.add(document)
  except:
    print("[WARNING] one discarded")

# Output
DB.to_disk(dev_spacy_path)

[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded
[WARNING] one discarded


Una vez tenemos el nuevo formato necesario por spaCy para el entrenamiento, podemos entrenar el modelo. Este modelo requiere de una gran configuración que se presenta en el fichero *spacy_config.cfg* que descargamos a continuación.

Entre los parámetros que se pueden configurar podemos destacar los del entrenamiento que se muestran a continuación.



```
[training]
dev_corpus = "corpora.dev"
train_corpus = "corpora.train"
seed = ${system.seed}
gpu_allocator = ${system.gpu_allocator}
dropout = 0.1
accumulate_gradient = 1
patience = 1600
max_epochs = 0
max_steps = 2200
eval_frequency = 200
frozen_components = []
annotating_components = []
before_to_disk = null
```

Podemos observar que se pueden configurar los corpus de **train** y **dev** que nosotros también pasamos como parámetro, el número de pasos, épocas, dropout, la frecuencia de evaluación, etc.


In [ ]:
new_model_spacy_dir = "spacy_ner_model/"
# Descargamos un fichero de configuración para el entrenamiento
!wget -N --no-check-certificate https://valencia.inf.um.es/valencia-plne/spacy_config.cfg



--2025-02-26 16:38:16--  https://valencia.inf.um.es/valencia-plne/spacy_config.cfg
Resolving valencia.inf.um.es (valencia.inf.um.es)... 155.54.204.133
Connecting to valencia.inf.um.es (valencia.inf.um.es)|155.54.204.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2602 (2.5K)
Saving to: ‘spacy_config.cfg’

spacy_config.cfg    100%[===================>]   2.54K  --.-KB/s    in 0s      

2025-02-26 16:38:17 (564 MB/s) - ‘spacy_config.cfg’ saved [2602/2602]



Una vez tenemos la configuración proceder con el entrenamiento.

En este caso indicamos en el script el conjunto de **train** y **dev** que no coinciden con los que están definidos en el fichero de configuración.

In [ ]:
# Ejecutamos el comando para el entrenamiento
!python -m spacy train spacy_config.cfg --paths.train train.spacy --paths.dev dev.spacy --output spacy_ner_model

✔ Created output directory: spacy_ner_model
ℹ Saving to output directory: spacy_ner_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    160.83    0.29    0.21    0.50    0.00
  0     200       1370.86   8248.04   23.50   25.40   21.86    0.23
  0     400        136.32   3470.95   40.21   54.91   31.72    0.40
  1     600        161.77   3259.25   35.33   54.67   26.10    0.35
  1     800        177.08   2830.50   45.30   60.74   36.12    0.45
  1    1000        183.96   2736.90   44.36   67.49   33.04    0.44
  2    1200        206.92   2742.08   52.01   60.06   45.87    0.52
  2    1400        254.32   2151.74   51.00   

###Inferencia

Una vez entrenado el modelo, el mejor modelo estará en la carpeta "model-best" dentro de la carpeta "spacy_ner_model"

In [ ]:
import spacy
import pandas as pd

# Cargar el modelo entrenado
spacy_nlp = spacy.load(f"{new_model_spacy_dir}/model-best")

En el siguiente código inferimos las mismas frases médicas y creamos otra columna para guardar las entidades identificadas por este nuevo modelo.

In [ ]:
# Inicializa nuevas columnas en el DataFrame con listas vacías
df['entidades_spacy'] = [[] for _ in range(len(df))]


# Procesa cada frase en el DataFrame
for index, row in df.iterrows():
    frase = row['texto']  # Suponiendo que la columna con las frases se llama 'texto'
    spaCyDoc = spacy_nlp(frase)

    entidades = []

    for ent in spaCyDoc.ents:
        entidades.append(ent.text)

    df.at[index, 'entidades_spacy'] = entidades

df.head(20)

,texto,entidades_stanza,entidades_spacy
0,El paciente presenta fiebre alta y escalofríos...,"[fiebre, escalofríos]","[fiebre alta, escalofríos]"
1,El paciente presenta dolor de cabeza intenso a...,[dolor de cabeza intenso acompañado de sensibi...,[dolor de cabeza intenso acompañado de sensibi...
2,"El paciente presenta tos seca persistente, por...",[tos seca persistente],[tos seca persistente]
3,"El paciente presenta dificultad para respirar,...",[dificultad para respirar],[dificultad para respirar]
4,El paciente presenta fatiga extrema y debilida...,"[fatiga extrema, debilidad muscular]","[fatiga, extrema, debilidad muscular]"
5,El paciente presenta dolor muscular generaliza...,"[dolor muscular generalizado, hidratación adec...",[dolor muscular generalizado]
6,"El paciente presenta congestión nasal severa, ...",[congestión nasal severa],[congestión nasal severa]
7,El paciente presenta pérdida del sentido del o...,[pérdida del sentido del olfato y del gusto],[pérdida del sentido del olfato y del gusto]
8,El paciente presenta sudoración nocturna recur...,[sudoración nocturna],"[sudoración nocturna, trastornos hormonales]"
9,"El paciente presenta dolor abdominal difuso, p...",[dolor abdominal difuso],[dolor abdominal difuso]
